In [1]:
import anndata as ad
import scanpy as sc
import numpy as np
import scipy as scp
import os
import re

In [2]:
#Read the downloaded output file names and combine file with same S
adataNames = os.listdir('./adataObjsKeep/')
adataList = []
adataNames.sort()
adataNames

['S13_L002_adata_keep.h5ad',
 'S13_L003_adata_keep.h5ad',
 'S13_L004_adata_keep.h5ad',
 'S13_L005_adata_keep.h5ad',
 'S13_L006_adata_keep.h5ad',
 'S13_L007_adata_keep.h5ad',
 'S13_L008_adata_keep.h5ad',
 'S14_L002_adata_keep.h5ad',
 'S14_L003_adata_keep.h5ad',
 'S14_L004_adata_keep.h5ad',
 'S14_L005_adata_keep.h5ad',
 'S14_L006_adata_keep.h5ad',
 'S14_L007_adata_keep.h5ad',
 'S14_L008_adata_keep.h5ad',
 'S15_L002_adata_keep.h5ad',
 'S15_L003_adata_keep.h5ad',
 'S15_L004_adata_keep.h5ad',
 'S15_L005_adata_keep.h5ad',
 'S15_L006_adata_keep.h5ad',
 'S15_L007_adata_keep.h5ad',
 'S15_L008_adata_keep.h5ad',
 'S16_L002_adata_keep.h5ad',
 'S16_L003_adata_keep.h5ad',
 'S16_L004_adata_keep.h5ad',
 'S16_L005_adata_keep.h5ad',
 'S16_L006_adata_keep.h5ad',
 'S16_L007_adata_keep.h5ad',
 'S16_L008_adata_keep.h5ad',
 'S17_L002_adata_keep.h5ad',
 'S17_L003_adata_keep.h5ad',
 'S17_L004_adata_keep.h5ad',
 'S17_L005_adata_keep.h5ad',
 'S17_L006_adata_keep.h5ad',
 'S17_L007_adata_keep.h5ad',
 'S17_L008_ada

In [3]:
# Save same sample indices in a dictionary
samples = []
sampleDict = {}
for i,an in enumerate(adataNames):
    match = re.search('S[0-9]+',an).group()
    samples.append(match)
    if match in sampleDict:
        sampleDict[match].append(i)
    else:
        sampleDict[match] = [i]
sampleDict

{'S13': [0, 1, 2, 3, 4, 5, 6],
 'S14': [7, 8, 9, 10, 11, 12, 13],
 'S15': [14, 15, 16, 17, 18, 19, 20],
 'S16': [21, 22, 23, 24, 25, 26, 27],
 'S17': [28, 29, 30, 31, 32, 33, 34],
 'S18': [35, 36, 37, 38, 39, 40, 41],
 'S1': [42, 43, 44, 45, 46, 47, 48, 49],
 'S2': [50, 51, 52, 53, 54, 55, 56],
 'S3': [57, 58, 59, 60, 61, 62, 63],
 'S4': [64, 65, 66, 67, 68, 69, 70],
 'S5': [71, 72, 73, 74, 75, 76, 77],
 'S6': [78, 79, 80, 81, 82, 83, 84]}

In [4]:
list(sampleDict.keys())

['S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6']

In [5]:
%%time
# The idea is to loop through each sample and then join them together
for s in list(sampleDict.keys())[4:]:
# for s in sampleDict.keys()
    print('file ' + s)
    adataList = []
    indices = sampleDict[s]
    for i in indices:
        adataList.append(ad.read_h5ad('./adataObjsKeep/'+ adataNames[i]))
    print(adataList)
    checkSameVar = []
    for i in range(1,len(adataList)):
        checkSameVar.append(all(adataList[i].var.index == adataList[i-1].var.index))
    check = all(checkSameVar)
    assert check == True, "The vars are not same in all adata"
    #convert each sparse matrix to array
    print('converting to array')
    XList = []
    SList = []
    UList = []
    obs = []
    var_names = list(adataList[0].var.index)
    obsNames = []
    for an in adataList:
        XList.append(an.X.toarray())
        SList.append(an.layers['spliced'].toarray())
        UList.append(an.layers['unspliced'].toarray())
        obsNames += list(an.obs.index.values)
    cellBarcodes = set(obsNames)
    rows = len(cellBarcodes)
    cols = adataList[0].shape[1]
    cellDict = dict(zip(cellBarcodes,range(rows)))
    X = np.zeros((rows,cols))
    S = np.zeros((rows,cols))
    U = np.zeros((rows,cols))
    print('adding values')
    for i in range(len(adataList)):
        for j in range(adataList[i].shape[0]):
            if j % 10000 == 0:
                print('done ' + str(j) + ' of file no ' + str(i+1) + ' of ' + str(len(adataList)))
            idx = cellDict[adataList[i].obs.index[j]]
            X[idx] = X[idx] + XList[i][j]
            S[idx] = S[idx] + SList[i][j]
            U[idx] = U[idx] + UList[i][j]
    print('converting into sparse matrices')
    X = scp.sparse.csr_matrix(X)
    S = scp.sparse.csr_matrix(S)
    U = scp.sparse.csr_matrix(U)
    concatAdata = ad.AnnData(X,
                  {'obs_names': cellBarcodes},
                  {'var_names': var_names},
                   layers={'spliced':S,
                          'unspliced':U})
    filename = './jointAdata/' + s + '.h5ad'
    print('joint adata is:')
    print(concatAdata)
    print('writing file ' + s)
    concatAdata.write(filename)

file S17
[AnnData object with n_obs × n_vars = 15696 × 60623 
    layers: 'spliced', 'unspliced', AnnData object with n_obs × n_vars = 11690 × 60623 
    layers: 'spliced', 'unspliced', AnnData object with n_obs × n_vars = 16747 × 60623 
    layers: 'spliced', 'unspliced', AnnData object with n_obs × n_vars = 11760 × 60623 
    layers: 'spliced', 'unspliced', AnnData object with n_obs × n_vars = 16282 × 60623 
    layers: 'spliced', 'unspliced', AnnData object with n_obs × n_vars = 17557 × 60623 
    layers: 'spliced', 'unspliced', AnnData object with n_obs × n_vars = 16973 × 60623 
    layers: 'spliced', 'unspliced']
converting to array
adding values
done 0 of file no 1 of 7
done 10000 of file no 1 of 7
done 0 of file no 2 of 7
done 10000 of file no 2 of 7
done 0 of file no 3 of 7
done 10000 of file no 3 of 7
done 0 of file no 4 of 7
done 10000 of file no 4 of 7
done 0 of file no 5 of 7
done 10000 of file no 5 of 7
done 0 of file no 6 of 7
done 10000 of file no 6 of 7
done 0 of file n

file S6
[AnnData object with n_obs × n_vars = 15494 × 60623 
    layers: 'spliced', 'unspliced', AnnData object with n_obs × n_vars = 11399 × 60623 
    layers: 'spliced', 'unspliced', AnnData object with n_obs × n_vars = 16409 × 60623 
    layers: 'spliced', 'unspliced', AnnData object with n_obs × n_vars = 11542 × 60623 
    layers: 'spliced', 'unspliced', AnnData object with n_obs × n_vars = 15812 × 60623 
    layers: 'spliced', 'unspliced', AnnData object with n_obs × n_vars = 16091 × 60623 
    layers: 'spliced', 'unspliced', AnnData object with n_obs × n_vars = 16788 × 60623 
    layers: 'spliced', 'unspliced']
converting to array
adding values
done 0 of file no 1 of 7
done 10000 of file no 1 of 7
done 0 of file no 2 of 7
done 10000 of file no 2 of 7
done 0 of file no 3 of 7
done 10000 of file no 3 of 7
done 0 of file no 4 of 7
done 10000 of file no 4 of 7
done 0 of file no 5 of 7
done 10000 of file no 5 of 7
done 0 of file no 6 of 7
done 10000 of file no 6 of 7
done 0 of file no

In [6]:
adat = ad.read_h5ad('jointAdata/S13.h5ad')
adat.X

<23216x60623 sparse matrix of type '<class 'numpy.float32'>'
	with 9208121 stored elements in Compressed Sparse Row format>

In [7]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('adat', 9471894),
 ('concatAdata', 9466492),
 ('an', 9002650),
 ('cellBarcodes', 2097376),
 ('cellDict', 1310816),
 ('obsNames', 931920),
 ('var_names', 545712),
 ('adataNames', 768),
 ('samples', 768),
 ('sampleDict', 648),
 ('indices', 136),
 ('SList', 128),
 ('UList', 128),
 ('XList', 128),
 ('adataList', 128),
 ('checkSameVar', 128),
 ('ad', 80),
 ('np', 80),
 ('sc', 80),
 ('scp', 80),
 ('filename', 69),
 ('obs', 64),
 ('S', 56),
 ('U', 56),
 ('X', 56),
 ('match', 51),
 ('s', 51),
 ('check', 28),
 ('cols', 28),
 ('i', 28),
 ('idx', 28),
 ('j', 28),
 ('rows', 28)]